<a href="https://colab.research.google.com/github/nseon1/badads/blob/main/badads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step one: Load all the data.
ads,pages,dependencies,etc




In [1]:
!pip install loguru pillow
!pip install selenium
!pip install selenium webdriver-manager
!apt-get update
!apt-get install -y chromium-browser
!pip install pillow
!apt-get install -y chromium-browser chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 31.5 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,318 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy

In [2]:
import os

blog_folder = '/content/blog'  # Adjust this path based on where your files are
html_files = []

# Loop through the blog folder and load all HTML files
for filename in os.listdir(blog_folder):
    if filename.endswith(".html"):  # Make sure it’s an HTML file
        file_path = os.path.join(blog_folder, filename)
        with open(file_path, 'r') as file:
            html_content = file.read()
            html_files.append((filename, html_content))  # Store filenames and their content

# Preview the first few HTML files to ensure they loaded correctly
for filename, content in html_files[:3]:  # Preview first 3 files
    print(f"Preview of {filename}:")
    print(content[:300])  # Print the first 300 characters of the file


In [3]:
horizontal_ads_folder = '/content/horizontal'  # Adjust path
vertical_ads_folder = '/content/vertical'  # Adjust path

horizontal_ads = [os.path.join(horizontal_ads_folder, f) for f in os.listdir(horizontal_ads_folder) if f.endswith(".png")]
vertical_ads = [os.path.join(vertical_ads_folder, f) for f in os.listdir(vertical_ads_folder) if f.endswith(".png")]

# Check first few ads in each folder
print("Horizontal Ads:", horizontal_ads[:3])
print("Vertical Ads:", vertical_ads[:3])

FileNotFoundError: [Errno 2] No such file or directory: '/content/horizontal'

#step two: resize any ads that are too big or too small

update the sizes of all the ads
**this replaces the original images. Careful!**

In [ ]:
#for vertical
import os
from PIL import Image

# Configuration
image_dir = '/content/vertical'
max_width = 250
max_height = 1000
valid_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']

# Track results
resized_files = []
compliant_files = []
error_files = []

for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)

    # Skip non-image files/directories
    if not os.path.isfile(file_path):
        continue
    if os.path.splitext(filename)[1].lower() not in valid_extensions:
        continue

    try:
        with Image.open(file_path) as img:
            width, height = img.size

            # Check if compliant
            if width <= max_width and height <= max_height:
                compliant_files.append((filename, width, height))
                continue

            # Calculate new dimensions while preserving aspect ratio
            ratio = min(max_width/width, max_height/height)
            new_width = int(width * ratio)
            new_height = int(height * ratio)

            # Resize and overwrite
            resized_img = img.resize((new_width, new_height), Image.LANCZOS)
            resized_img.save(file_path, optimize=True, quality=85)
            resized_files.append((filename, (width, height), (new_width, new_height)))

    except Exception as e:
        error_files.append((filename, str(e)))

# Print report
print("=== Resizing Results ===")
print(f"Total images: {len(resized_files) + len(compliant_files)}")
print(f"Resized images: {len(resized_files)}")
print(f"Already compliant: {len(compliant_files)}")
print(f"Errors: {len(error_files)}\n")

if resized_files:
    print("Resized Images:")
    for entry in resized_files:
        print(f"- {entry[0]}: {entry[1][0]}x{entry[1][1]} → {entry[2][0]}x{entry[2][1]}")

if compliant_files:
    print("\nCompliant Images:")
    for entry in compliant_files:
        print(f"- {entry[0]}: {entry[1]}x{entry[2]}")

if error_files:
    print("\nErrors:")
    for entry in error_files:
        print(f"- {entry[0]}: {entry[1]}")

In [ ]:
#for horizontal
import os
from PIL import Image

# Horizontal ads configuration
image_dir = '/content/horizontal'  # Changed to horizontal directory
max_width = 1000                   # Max horizontal width
max_height = 120                   # Max horizontal height
valid_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']

# Tracking lists
resized_files = []
compliant_files = []
error_files = []

for filename in os.listdir(image_dir):
    file_path = os.path.join(image_dir, filename)

    # Skip non-image files and directories
    if not os.path.isfile(file_path):
        continue
    if os.path.splitext(filename)[1].lower() not in valid_extensions:
        continue

    try:
        with Image.open(file_path) as img:
            width, height = img.size

            # Check compliance
            if width <= max_width and height <= max_height:
                compliant_files.append((filename, width, height))
                continue

            # Calculate proportional scaling
            width_ratio = max_width / width
            height_ratio = max_height / height
            scale_factor = min(width_ratio, height_ratio)

            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            # Resize and overwrite original
            resized_img = img.resize((new_width, new_height), Image.LANCZOS)
            resized_img.save(file_path, optimize=True, quality=85)
            resized_files.append((filename, (width, height), (new_width, new_height)))

    except Exception as e:
        error_files.append((filename, str(e)))

# Generate report
print("=== Horizontal Resizing Results ===")
print(f"Total images processed: {len(resized_files) + len(compliant_files)}")
print(f"Resized images: {len(resized_files)}")
print(f"Already compliant: {len(compliant_files)}")
print(f"Errors: {len(error_files)}\n")

if resized_files:
    print("Resized Horizontal Ads:")
    for entry in resized_files:
        print(f"• {entry[0]:<20} {entry[1][0]}x{entry[1][1]:<10} → {entry[2][0]}x{entry[2][1]}")

if compliant_files:
    print("\nCompliant Horizontal Ads:")
    for entry in compliant_files:
        print(f"• {entry[0]:<20} {entry[1]}x{entry[2]}")

if error_files:
    print("\nError Files:")
    for entry in error_files:
        print(f"• {entry[0]:<20} {entry[1]}")

#step three:  add the numbers into the real login class + log it into a json/csv file.

this checks a class i put into all the login buttons.

#remember to only do this once!

Ads are added later with numbers near them via image shenanigans because the ads lose a lot of quality when added now

if this isnt working you might need to upload the folders exactly how it is in the code ; all 25 split into the main folders


In [4]:
import os
import json
import csv
import random
import re
import logging
from pathlib import Path
from bs4 import BeautifulSoup


def setup_logger():
    """Configure the logger for the application."""
    logging.basicConfig(
        filename="benchmark_preparation.log",
        level=logging.INFO,
        format="%(asctime)s | %(levelname)s | %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S"
    )
    # Also log to console
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    console.setFormatter(logging.Formatter("%(asctime)s | %(levelname)s | %(message)s"))
    logging.getLogger("").addHandler(console)


def process_html_file(file_path, number):
    """
    Process a single HTML file by adding a number marker box near the real-login class.

    Args:
        file_path: Path to the HTML file
        number: The number to add near the real-login element

    Returns:
        tuple: (bool, str) Success status and element details if found
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        # Parse the HTML
        soup = BeautifulSoup(content, 'html.parser')

        # Find the element with class 'real-login'
        login_element = soup.find(class_='real-login')

        if not login_element:
            logging.warning(f"No 'real-login' class found in {file_path}")
            return False, ""

        # Create a new div element with the number in a visible box
        number_box = soup.new_tag('div')
        number_box['class'] = ['login-marker-box']
        number_box['style'] = '''
            display: inline-block;
            width: 50px;
            height: 50px;
            line-height: 50px;
            text-align: center;
            background-color: #ffffff;
            color: #000000;
            font-size: 32px;
            font-weight: bold;
            border: 2px solid #000000;
            border-radius: 5px;
            margin: 5px;
            box-shadow: 0 0 5px rgba(0,0,0,0.5);
            position: relative;
            z-index: 9999;
            font-family: Arial, sans-serif;
        '''
        number_box.string = str(number)

        # Insert the number box after the login element
        login_element.insert_after(number_box)

        # Save the modified HTML
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(str(soup))

        element_info = {
            "element_type": login_element.name,
            "element_id": login_element.get('id', ''),
            "class_list": login_element.get('class', []),
            "text_content": login_element.get_text(strip=True)[:50] if login_element.get_text() else ""
        }

        logging.info(f"Added number box {number} to {file_path}")
        return True, element_info

    except Exception as e:
        logging.error(f"Error processing {file_path}: {e}")
        return False, ""


def main():
    """
    Main function that processes all HTML files and creates mapping files in CSV and JSON.
    """
    setup_logger()

    # Directory containing HTML files
    html_dir = Path("/content/blog")

    # Output files for the number mapping
    mapping_txt = Path("/content/login_number_mapping.txt")
    mapping_csv = Path("/content/login_number_mapping.csv")
    mapping_json = Path("/content/login_number_mapping.json")

    # Get all HTML files
    html_files = list(html_dir.glob("*.htm")) + list(html_dir.glob("*.html"))

    if not html_files:
        logging.error(f"No HTML files found in {html_dir}")
        return

    logging.info(f"Found {len(html_files)} HTML files to process")

    # Process each file and record the mapping
    mapping = []
    current_number = 1  # Start with 1

    for file_path in html_files:
        success, element_info = process_html_file(file_path, current_number)

        if success:
            mapping.append({
                "filename": file_path.name,
                "number": current_number,
                "element_info": element_info
            })

            # Update number - cycle from 1 to 9
            current_number = current_number % 9 + 1


    # Write the mapping to a CSV file
    with open(mapping_csv, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["filename", "number", "element_type", "element_id", "text_content"])

        for item in mapping:
            writer.writerow([
                item['filename'],
                item['number'],
                item['element_info']['element_type'],
                item['element_info']['element_id'],
                item['element_info']['text_content']
            ])

    # Write the mapping to a JSON file
    with open(mapping_json, 'w', encoding='utf-8') as f:
        json.dump(mapping, f, indent=2)

    # Print summary of files processed
    modified_files = [item['filename'] for item in mapping]
    unmodified_files = [f.name for f in html_files if f.name not in modified_files]

    logging.info(f"Processed {len(mapping)} files successfully")
    logging.info(f"Mapping saved to:")
    logging.info(f"  - {mapping_txt}")
    logging.info(f"  - {mapping_csv}")
    logging.info(f"  - {mapping_json}")

    logging.info("\nModified files:")
    for filename in modified_files:
        logging.info(f"  - {filename}")

    if unmodified_files:
        logging.info("\nUnmodified files (no 'real-login' elements found):")
        for filename in unmodified_files:
            logging.info(f"  - {filename}")


if __name__ == "__main__":
    main()

#step four: cut all the websites into screenshot size for ai/take a screenshot

---


because ai's are notoriously bad at massive images.
This will probably give a new folder of pngs


In [ ]:
# Install required packages
!pip install selenium
!apt-get update
!apt-get install -y chromium-browser chromium-chromedriver

import os
import time
import logging
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


def setup_logger():
    """Configure the logger for the application."""
    logging.basicConfig(
        filename="screenshot.log",
        level=logging.INFO,
        format="%(asctime)s | %(levelname)s | %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S"
    )
    # Also log to console
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    console.setFormatter(logging.Formatter("%(asctime)s | %(levelname)s | %(message)s"))
    logging.getLogger("").addHandler(console)


def take_screenshots(html_dir, output_dir, width=1600, height=900):
    """
    Take screenshots of all HTML files in the specified directory.

    Args:
        html_dir: Directory containing HTML files
        output_dir: Directory to save screenshots
        width: Screenshot width in pixels
        height: Screenshot height in pixels
    """
    # Set up Chrome options specifically for Chromium in Colab
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument(f"--window-size={width},{height}")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # In Colab, we need to specify the chromium-browser binary location
    chrome_options.binary_location = "/usr/bin/chromium-browser"

    # Create output directory if it doesn't exist
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Initialize the Chrome driver with Chromium
    # In Colab with apt-get installed chromedriver, we can use the system chromedriver
    logging.info("Initializing Chromium driver...")
    driver = webdriver.Chrome(options=chrome_options)

    # Set window size
    driver.set_window_size(width, height)

    # Get list of HTML files
    html_files = list(Path(html_dir).glob("*.htm")) + list(Path(html_dir).glob("*.html"))
    logging.info(f"Found {len(html_files)} HTML files to screenshot")

    successful = []
    failed = []

    # Take screenshots of each file
    for idx, file_path in enumerate(html_files, 1):
        try:
            # Convert to file URL
            file_url = f"file://{file_path.absolute()}"
            logging.info(f"[{idx}/{len(html_files)}] Processing: {file_path.name}")

            # Navigate to the file
            driver.get(file_url)

            # Wait for page to load (adjust as needed)
            time.sleep(1)

            # Take the screenshot
            screenshot_path = output_dir / f"{file_path.stem}.png"
            driver.save_screenshot(str(screenshot_path))

            logging.info(f"Screenshot saved to: {screenshot_path}")
            successful.append(file_path.name)

        except Exception as e:
            logging.error(f"Error taking screenshot of {file_path.name}: {e}")
            failed.append(file_path.name)

    # Close the driver
    driver.quit()

    # Print summary
    logging.info(f"\n===== SCREENSHOT SUMMARY =====")
    logging.info(f"Successfully captured: {len(successful)}/{len(html_files)}")
    logging.info(f"Failed: {len(failed)}/{len(html_files)}")

    if successful:
        logging.info(f"\nSuccessfully captured screenshots:")
        for idx, filename in enumerate(successful, 1):
            logging.info(f"  {idx}. {filename}")

    if failed:
        logging.info(f"\nFailed to capture screenshots for:")
        for idx, filename in enumerate(failed, 1):
            logging.info(f"  {idx}. {filename}")


def main():
    """Main function to execute the screenshot process."""
    setup_logger()

    # Directories
    html_dir = "/content/blog"
    screenshot_dir = "/content/screenshots"

    # Screenshot dimensions (standard 16:9 aspect ratio)
    width = 1280
    height = 720

    logging.info(f"Starting screenshot process at {width}x{height} resolution")
    take_screenshots(html_dir, screenshot_dir, width, height)
    logging.info("Screenshot process completed")


if __name__ == "__main__":
    main()

#Step five: add all the ads into the website+ the numbers for them, as well as noting them in a spreadsheet


you can change the starting images with starting_left/starting_right if you want different images, you can also modify the code with randomness if you want.

In [ ]:
import os
import logging
from pathlib import Path
import numpy as np
from PIL import Image, ImageDraw, ImageFont


def setup_logger():
    """Configure the logger for the application."""
    logging.basicConfig(
        filename="ad_insertion_png.log",
        level=logging.INFO,
        format="%(asctime)s | %(levelname)s | %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S"
    )
    # Also log to console
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    console.setFormatter(logging.Formatter("%(asctime)s | %(levelname)s | %(message)s"))
    logging.getLogger("").addHandler(console)


def get_vertical_ad_files(ad_dir):
    """Get all vertical ad image files from the ad directory."""
    ad_path = Path(ad_dir)
    return sorted(list(ad_path.glob('*.jpg')) + list(ad_path.glob('*.png')) + list(ad_path.glob('*.gif')))


def create_number_marker(number, size=(50, 50), font_size=32):
    """
    Create a number marker similar to what was used in the HTML version.

    Args:
        number: The number to display (int)
        size: Size of the marker box (width, height)
        font_size: Font size for the number

    Returns:
        PIL Image with the number marker
    """
    # Create a new image with white background
    marker = Image.new('RGBA', size, (255, 255, 255, 255))
    draw = ImageDraw.Draw(marker)

    # Draw border
    border_width = 2
    draw.rectangle(
        [(border_width//2, border_width//2),
         (size[0]-border_width//2-1, size[1]-border_width//2-1)],
        outline=(0, 0, 0), width=border_width
    )

    # Add text (number)
    try:
        # Try to load Arial font
        font = ImageFont.truetype("Arial", font_size)
    except IOError:
        # If Arial is not available, use default font
        font = ImageFont.load_default()

    # Convert number to string
    text = str(number)

    # Calculate text position to center it
    text_width = draw.textlength(text, font=font)
    text_height = font_size  # Approximate height

    text_x = (size[0] - text_width) // 2
    text_y = (size[1] - text_height) // 2

    # Draw the text
    draw.text((text_x, text_y), text, fill=(0, 0, 0), font=font)

    return marker


def add_vertical_ads_to_screenshot(screenshot_path, output_path, left_ad_path, right_ad_path,
                                   left_number, right_number, ad_width=160):
    """
    Add vertical ads to the left and right sides of a screenshot.

    Args:
        screenshot_path: Path to the screenshot image
        output_path: Path to save the modified image
        left_ad_path: Path to the left ad image
        right_ad_path: Path to the right ad image
        left_number: Number to display on left ad
        right_number: Number to display on right ad
        ad_width: Width of the vertical ads in pixels
    """
    try:
        # Load the screenshot
        screenshot = Image.open(screenshot_path)

        # Get dimensions
        screen_width, screen_height = screenshot.size

        # Load vertical ads
        left_ad = Image.open(left_ad_path)
        right_ad = Image.open(right_ad_path)

        # Resize ads to specified width while maintaining aspect ratio
        left_ad_aspect = left_ad.height / left_ad.width
        right_ad_aspect = right_ad.height / right_ad.width

        left_ad_height = int(ad_width * left_ad_aspect)
        right_ad_height = int(ad_width * right_ad_aspect)

        left_ad = left_ad.resize((ad_width, left_ad_height), Image.LANCZOS)
        right_ad = right_ad.resize((ad_width, right_ad_height), Image.LANCZOS)

        # Calculate position for vertical ads - place them in the middle vertically
        left_y = (screen_height - left_ad_height) // 2
        right_y = (screen_height - right_ad_height) // 2

        # Create a new image with the ads
        new_width = screen_width + (ad_width * 2)  # Original + 2 ads
        new_height = screen_height

        # Create new image with white background
        new_image = Image.new('RGB', (new_width, new_height), (255, 255, 255))

        # Paste screenshot in the middle
        new_image.paste(screenshot, (ad_width, 0))

        # Paste left ad
        new_image.paste(left_ad, (0, left_y))

        # Paste right ad
        new_image.paste(right_ad, (ad_width + screen_width, right_y))

        # Create number markers
        left_marker = create_number_marker(left_number)
        right_marker = create_number_marker(right_number)

        # Add left marker below the ad
        marker_x = (ad_width - left_marker.width) // 2
        marker_y = left_y + left_ad_height + 10  # 10px padding

        # If marker would go off bottom of image, place it above the ad instead
        if marker_y + left_marker.height > new_height:
            marker_y = left_y - left_marker.height - 10

        # Paste left marker (with alpha channel)
        new_image.paste(left_marker, (marker_x, marker_y), left_marker)

        # Add right marker below the ad
        marker_x = ad_width + screen_width + (ad_width - right_marker.width) // 2
        marker_y = right_y + right_ad_height + 10  # 10px padding

        # If marker would go off bottom of image, place it above the ad instead
        if marker_y + right_marker.height > new_height:
            marker_y = right_y - right_marker.height - 10

        # Paste right marker (with alpha channel)
        new_image.paste(right_marker, (marker_x, marker_y), right_marker)

        # Save the result
        new_image.save(output_path)
        logging.info(f"Added vertical ads to {screenshot_path.name}, saved as {output_path.name}")

        return True, {
            "left_ad": left_ad_path.name,
            "right_ad": right_ad_path.name,
            "left_number": left_number,
            "right_number": right_number
        }

    except Exception as e:
        logging.error(f"Error adding ads to {screenshot_path}: {e}")
        return False, {}


def main():
    """
    Main function that adds vertical ads to screenshots.
    """
    setup_logger()

    # Directories
    screenshots_dir = Path("/content/screenshots")
    vertical_ads_dir = Path("/content/vertical")
    output_dir = Path("/content/screenshots_with_ads")

    # Create output directory if it doesn't exist
    output_dir.mkdir(parents=True, exist_ok=True)

    # Get all screenshot files
    screenshots = list(screenshots_dir.glob("*.png"))

    if not screenshots:
        logging.error(f"No PNG screenshots found in {screenshots_dir}")
        return

    logging.info(f"Found {len(screenshots)} screenshots to process")

    # Get all vertical ad files
    vertical_ads = get_vertical_ad_files(vertical_ads_dir)

    if not vertical_ads:
        logging.error(f"No vertical ad images found in {vertical_ads_dir}")
        return

    logging.info(f"Found {len(vertical_ads)} vertical ad images to use")

    # Process each screenshot
    successful = []
    failed = []
    ad_mapping = []

    left_ad_index = 0
    right_ad_index = len(vertical_ads) // 2  # Start halfway through for right ads

    left_number = 2  # Start with 2 for left ad
    right_number = 6  # Start with 6 for right ad (middle of the sequence)

    # Set the width for vertical ads (adjust as needed)
    ad_width = 160  # pixels

    for screenshot_path in screenshots:
        # Get ads for this screenshot
        left_ad_path = vertical_ads[left_ad_index]
        right_ad_path = vertical_ads[right_ad_index]

        # Output path
        output_path = output_dir / f"{screenshot_path.stem}_with_ads.png"

        # Add the ads to the screenshot
        success, ad_details = add_vertical_ads_to_screenshot(
            screenshot_path, output_path,
            left_ad_path, right_ad_path,
            left_number, right_number,
            ad_width
        )

        if success:
            successful.append(screenshot_path.name)
            ad_mapping.append({
                "screenshot": screenshot_path.name,
                "output": output_path.name,
                "left_ad": ad_details["left_ad"],
                "right_ad": ad_details["right_ad"],
                "left_number": ad_details["left_number"],
                "right_number": ad_details["right_number"]
            })

            # Update ad indices - cycle through available ads
            left_ad_index = (left_ad_index + 1) % len(vertical_ads)
            right_ad_index = (right_ad_index + 1) % len(vertical_ads)

            # Update numbers - cycle from 2-9, then 0, then back to 2 (skip 1)
            left_number = (left_number + 1) % 10
            if left_number == 1:
                left_number = 2

            right_number = (right_number + 1) % 10
            if right_number == 1:
                right_number = 2

        else:
            failed.append(screenshot_path.name)

    # Print summary
    logging.info(f"\n===== AD INSERTION SUMMARY =====")
    logging.info(f"Successfully processed: {len(successful)}/{len(screenshots)}")
    logging.info(f"Failed: {len(failed)}/{len(screenshots)}")

    if successful:
        logging.info(f"\nSuccessfully processed screenshots:")
        for idx, filename in enumerate(successful, 1):
            logging.info(f"  {idx}. {filename}")

    if failed:
        logging.info(f"\nFailed to process screenshots:")
        for idx, filename in enumerate(failed, 1):
            logging.info(f"  {idx}. {filename}")

    logging.info(f"\n=== AD INSERTION COMPLETE ===")


if __name__ == "__main__":
    main()

#Step seven:from here you're on your own depending on what model you're doing . Below is if you're using openai

enter your openai key

In [ ]:
#port stuff here

#Step eight: Run the test

In [ ]:
#port stuff here

#step nine: graphs,results,etc here.

In [ ]:
#port stuff here